-use the variables and zip text files to modify output


In [1]:
import pandas as pd
import requests
import json

In [2]:
#create list of zipcodes from txt file with zips
zips = open('chosen_zips.txt', 'r').readlines()
zips = [z.replace('\n', '') for z in zips]
zips = ','.join(zips)

In [3]:
#census api key
apiKey = ""

In [4]:
#same thing with variables
variables = open('variables_2013.txt', 'r').readlines()
variables = [z.replace('\n', '') for z in variables]
variables = ','.join(variables)


In [5]:
#just to check for errors, if a ',' appears at the end you need to delete the last blank line in the txt file
print(variables)
#the first variable here corresponds to the last value in columnNames

DP02_0066PE,DP04_0132E,DP04_0107E,DP04_0100E,DP04_0090PE,DP04_0045PE,DP02_0014PE,DP02_0014E,DP02_0013PE,DP02_0013E,DP02_0020PE,DP02_0020E,DP02_0010PE,DP02_0010E,DP02_0003PE,DP02_0003E,DP02_0001E,DP02_0016E,DP02_0015E,DP04_0088E,DP03_0062E,DP03_0119PE,DP03_0009PE,DP02_0065PE,DP02_0065E,DP02_0064PE,DP02_0064E,DP02_0061PE,DP02_0061E,DP05_0017E,DP05_0025E,DP02_0055E,DP05_0018E,DP05_0004E,DP05_0001E


In [6]:
#name your variables, 'zipcode' stays unchanged
#again, note that columnNames is flipped
columnNames = ['zipcode', 'pop', 'under5', '18plus', 'elemschool', '65plus', 'med_age', 'hschool', 'p_hschool', 'bachelors', 'p_bachelors', 'advanced', 'p_advanced', 'unemp',
               'pov', 'med_hholdinc', 'med_homeval', 'avg_hhold', 'avg_fam', 'hhold', 'famhhold', 'p_famhhold', '!famhhold', 'p_!famhhold', 'childpop', 'p_childpop', 'hholdunder18',
               'p_hholdunder18', 'hhold65plus', 'p_hhold65plus', 'p_owneroccup', 'p_oomortgages', 'med_smocmortgages', 'med_smoc!mortgages', 'med_rent', 'p_hschoolplus']
               

In [7]:
#base api, this is where you change the year and dataset
#it could be cleaned a little more with %s
#note it is currently the 2013/acs/acs5/profile dataset
baseAPI = "https://api.census.gov/data/2013/acs/acs5/profile?key=%s&get=%s&for=zip%%20code%%20tabulation%%20area:%s"

In [8]:
#custom API constructed
calledAPI = baseAPI % (apiKey, variables, zips)

In [9]:
#call API and collect the response
response = requests.get(calledAPI)

In [2]:
#check if all the variables are valid, if ok it will just print, if not there will be an error
#print(response.text)

In [11]:
#load the response into a JSON, ignoring the first element which is just field labels
formattedResponse = json.loads(response.text)[1:]

In [12]:
#flip the order of the response from [population, zipcode] -> [zipcode, population]
#this is kind of annoying since it affects the entire column name order
formattedResponse = [item[::-1] for item in formattedResponse]

In [13]:
#store the response in a dataframe
#remember the last column name is assigned to the first line in the variable text file
df = pd.DataFrame(columns=columnNames, data=formattedResponse)

In [14]:
df.head()

,zipcode,pop,under5,18plus,elemschool,65plus,med_age,hschool,p_hschool,bachelors,...,hholdunder18,p_hholdunder18,hhold65plus,p_hhold65plus,p_owneroccup,p_oomortgages,med_smocmortgages,med_smoc!mortgages,med_rent,p_hschoolplus
0,07059,15574,827,11217,2025,2176,43.3,1857,18.0,3557,...,2045,41.6,1507,30.6,91.6,64.5,3632,1001,1570,96.6
1,07060,44793,3215,34046,4662,3940,33.8,8915,30.3,3548,...,5605,40.3,2877,20.7,44.4,78.3,2494,1001,1156,74.1
2,07062,12571,866,9585,1251,1474,37.7,2938,35.0,1461,...,1605,39.5,1072,26.4,64.3,76.2,2385,1001,1216,83.9
3,07063,14552,1190,10864,1651,1229,33.4,3213,34.7,1049,...,2000,52.3,864,22.6,65.5,81.4,2434,1001,1328,73.5
4,07069,6130,316,4882,521,1338,46.9,583,13.2,1228,...,655,29.7,891,40.3,88.4,65.1,3866,1001,2001,92.5


In [1]:
#check dtypes
#df.dtypes
#convert as neccisary

In [16]:
#converting them to_numeric
#here I am converting everything after zipcode, the first column
colstoconvert = columnNames[1:]
df[colstoconvert] = df[colstoconvert].apply(pd.to_numeric, errors='coerce')


In [18]:
#not used but here for reference
#converting zipcode back to string
#somersetZipPopulations['zipcode'] = somersetZipPopulations['zipcode'].astype(str)
#somersetZipPopulations.dtypes

Now let's add the columns for the percentage of people 65 and older (p65+) and for school age children 5-17 (pSchoolAge).  Because the variables are different, SchoolAge requires different calculations than acs5_2018  (disregard this, I changed it so they both use "18 and older", called 'over18')

In [19]:
#first the p65+
df['p_65plus'] = (df['65plus']/df['pop'])

In [20]:
#now SchoolAge and then pSchoolAge
df['schoolage'] = df['pop'] - df['18plus']
df['schoolage'] = df['schoolage'] - df['under5']
df['p_schoolage'] = (df['schoolage'] / df['pop'])

#percent without a high school diploma
df['p_!hschool'] = df['p_hschoolplus'].apply(lambda x: 100-x)

#add a year column
df['year'] = '2013'

In [21]:
#convert whole number percentage columns to decimal so excel can apply percent formatting
pcolumns = ['p_hschool', 'p_bachelors', 'p_advanced', 'unemp', 'pov', 'p_famhhold', 'p_!famhhold', 'p_childpop',
            'p_hholdunder18', 'p_hhold65plus', 'p_owneroccup', 'p_oomortgages', 'p_hschoolplus', 'p_!hschool']

temp = df[pcolumns].apply(lambda x: x/100)

df = df.drop(pcolumns, axis=1)

df = df.join(temp)

In [22]:
#assign the names of towns and such
name_df = pd.read_excel("town_names.xlsx", dtype={'zipcode': object})

df = df.set_index('zipcode').join(name_df.set_index('zipcode'))

In [23]:
#save that dataframe to a CSV spreadsheet
df.to_excel('./acs5_data/acs5_2013_zip.xlsx', index=True)